This is going to be a script to split up an image into multiple images so
that DDC can act on them in //.

The target number of localizations is 4000 (or whatever min_loc is)
and there will be some buffer added to each side of the images to avoid boundary
effects. -CHB 2019

In [ ]:
import numpy as np
import random
import scipy.io as sio
import matplotlib.pyplot as plt
from tkinter import Tk
from tkinter.filedialog import askopenfilename
import os
import math
import time
from mpl_toolkits.mplot3d import Axes3D  # Required for 3D plotting

def make_into_cell(arr: np.ndarray) -> np.ndarray:
    """Put an array inside an array to consider it a cell in .mat."""
    container: np.ndarray = np.empty((1, 1), dtype=object)
    container[0, 0] = arr
    return container

Importing file, arrange them in proper list

In [ ]:

# Initialize variables
TrueLocalizations = []
Photons = []
Resolution = []
min_loc = 2000

# Use tkinter file dialog to mimic uigetfile
Tk().withdraw()
filename_full = askopenfilename(filetypes=[("MAT files", "*.mat")], title="Select HMM .mat file")
if not filename_full:
    print('Error! No (or wrong) file selected!')
    exit()

# Separate pathname and filename
pathname, filename_only = os.path.split(filename_full)

# This will load in all of the localizations from your structure file
full_filename = os.path.join(pathname, filename_only)
try:
    data = sio.loadmat(full_filename, squeeze_me=True, struct_as_record=False)
except Exception as e:
    print(f"Error loading MAT file: {e}")
    exit()

# Better handling of loaded data - handle both lists and numpy arrays
if 'LocalizationsFinal' in data:
    #check if there is only one ROI or not
    if type(data['LocalizationsFinal'][0][0]) == np.ndarray:
        LocalizationsFinal = data['LocalizationsFinal']
    else:
        LocalizationsFinal = np.array([data['LocalizationsFinal']])
else:
    LocalizationsFinal = []

if 'Frame_Information' in data:
    #check if there is only one ROI or not
    if type(data['Frame_Information'][0]) == np.ndarray:
        Frame_Information = data['Frame_Information']
    else:
        Frame_Information = np.array([data['Frame_Information']])
else:
    Frame_Information = []

if 'TrueLocalizations' in data:
    #check if there is only one ROI or not
    if type(data['TrueLocalizations'][0][0]) == np.ndarray:
        TrueLocalizations = data['TrueLocalizations']
    else:
        TrueLocalizations = np.array([data['TrueLocalizations']])
else:
    TrueLocalizations = []

if 'Photons' in data:
    #check if there is only one ROI or not
    if type(data['Photons'][0]) == np.ndarray:
        Photons = data['Photons']
    else:
        Photons = np.array([data['Photons']])
else:
    Photons = []

if 'Resolution' in data:
    Resolution = data['Resolution']
else:
    Resolution = []

Condition = filename_only



In [ ]:
len(TrueLocalizations)
TrueLocalizations
TrueLocalizations.shape[1] == 2

Adding zero to third coòumns if it's not exist

In [ ]:
# If photons cell does not exist, generate one
if len(Photons) == 0 or Photons is None:
    Photons = []  # reinitialize as an empty list
    for i in range(len(Frame_Information)):
        # Add array of one to the list for each frame in photons
        Photons.append(np.ones((len(Frame_Information[i]), 1), dtype=int))


# Initialize arrays to store split up images
LocalizationsFinal_Split = []
TrueLocalizations_Split = []
Frame_Information_Split = []
Photons_Split = []
Came_from_image = []
Parameters_to_split = []
temp_numb_of_loc = []
addonarray = []
cut1array = []
cut2array = []
cut3array = []

# Just in case you do not actually know the true localizations
if len(TrueLocalizations) == 0:
    TrueLocalizations = []
    for ijk in range(len(Frame_Information)):
        TrueLocalizations.append(np.empty((0, 3)))  # Empty array with 3 columns

# Process the localizations to ensure 3D format
processed_localizations = []
processed_true_localizations = []

# Print debug info about loaded data
print(f"Number of frames: {len(LocalizationsFinal)}")
print(LocalizationsFinal)


# Adding zero to the z coordinate if it is not present
for sdfv in range(len(LocalizationsFinal)):
    # Debug info
    print(f"Processing frame {sdfv+1}: ", end="")
    
    # Convert to numpy array
    try:
        if not isinstance(LocalizationsFinal[sdfv], np.ndarray):
            LF = np.array(LocalizationsFinal[sdfv], dtype=float)
        else:
            LF = LocalizationsFinal[sdfv].copy()
        
        
        print(f"shape: {LF.shape if hasattr(LF, 'shape') else 'scalar'}, size: {LF.size if hasattr(LF, 'size') else 1}")
        
        # Handle different shapes
        if LF.shape[1] == 2:
            processed_localizations.append(
                np.column_stack((
                    LF,
                    np.zeros((LF.shape[0], 1))
                    ))
                )
        else:
            # Already has 3+ columns, use first 3
            processed_localizations.append(LF[:, :3])
            print(f"Handled 2D array with final shape {processed_localizations[-1].shape}")
    except Exception as e:
        print(f"Error processing LocalizationsFinal for frame {sdfv}: {e}")
        processed_localizations.append(np.empty((0, 3)))
    
    # Do the same for TrueLocalizations
    try:
        if not isinstance(TrueLocalizations[sdfv], np.ndarray):
            TL = np.array(TrueLocalizations[sdfv], dtype=float)
        else:
            TL = TrueLocalizations[sdfv].copy()
        
        
        # Handle different shapes
        if TL.shape[1] == 2:
            processed_true_localizations.append(
                np.column_stack((
                    TL,
                    np.zeros((TL.shape[0], 1))
                    ))
                )
        else:
            # Already has 3+ columns, use first 3
            processed_true_localizations.append(TL[:, :3])
            print(f"Handled 2D array with final shape {processed_true_localizations[-1].shape}")
    except Exception as e:
        print(f"Error processing TrueLocalizations for frame {sdfv}: {e}")
        processed_true_localizations.append(np.empty((0, 3)))

# Replace the original lists with processed ones
LocalizationsFinal = processed_localizations
if len(processed_true_localizations) > 0:
    TrueLocalizations = processed_true_localizations


In [ ]:
processed_true_localizations

Check if input of Matlab and Python are same before main loop

In [ ]:
# Step 1: Transpose to (4000, 24, 3)
arr_transposed = np.array(LocalizationsFinal).transpose(1, 0, 2)
# Step 2: Reshape to (4000, 24*3)
Localization_final_reshaped = arr_transposed.reshape(len(LocalizationsFinal[0]), len(LocalizationsFinal) * len(LocalizationsFinal[0][0]))
np.savetxt('output-python.txt', Localization_final_reshaped , delimiter='\t')

In [ ]:
# Check thorugh pandas
import pandas as pd
df_python=pd.DataFrame(Localization_final_reshaped)
df_matlab=pd.read_csv('output.txt', sep='\t', header=None) 

#Calculate the difference between the two dataframes and check if it is zero
df_diff = df_python - df_matlab
np.any(df_diff>0.1)
np.any(df_diff<-0.1)

In [ ]:
# Main processing loop
counter = 0
for ksu in range(2):
    print(f"Processing frame {ksu+1}/{len(Frame_Information)}")
    
    # Get the localization data
    LF = LocalizationsFinal[ksu]
    
    # Extract columns - now we know LF has 3 columns
    X1, X2, X3 = LF[:,0], LF[:,1], LF[:,2]
    
    # Handle TrueLocalizations for this frame
    has_true_loc = (ksu < len(TrueLocalizations) and 
                    TrueLocalizations[ksu].size > 0)
    
    if has_true_loc:
        TL = TrueLocalizations[ksu]
        X1t, X2t, X3t = TL[:,0], TL[:,1], TL[:,2]
    
    X4 = Frame_Information[ksu].copy()
    addon = 250  # Buffer region
    
    # Phase space search
    scorefinal = np.inf
    onwers = 1

    ppf , ppf2, ppf3 = 0.2, 0.5, 1
    
    # Set parameters based on results
    pp, pp2, pp3 = ppf, ppf2, ppf3
    
    if len(X1) < min_loc:
        ppf = pp = ppf2 = pp2 = ppf3 = pp3 = 1
    
    flag = 0


    #Now that we know the optimum way to split up our specific image, we
    #now split it up. 
    pp = ppf
    pp2 = ppf2
    pp3 = ppf3

    if len(X1) < min_loc:
        ppf = 1
        ppf2 = 1
        ppf3 = 1
        pp = ppf
        pp2 = ppf2
        pp3 = ppf3

    flag = 0

    for i in range(1, math.ceil(1/pp) + 1):
        if flag == 1:
            break
        
        if i*pp < 1:
            cut1 = np.quantile(X1, [(i-1)*pp, i*pp])
        else:
            cut1 = np.quantile(X1, [(i-1)*pp, 1])
        
        for ii in range(1, math.ceil(1/pp2) + 1):
            if ii*pp2 < 1:
                cut2 = np.quantile(X2, [(ii-1)*pp2, ii*pp2])
            else:
                cut2 = np.quantile(X2, [(ii-1)*pp2, 1])
            
            for iii in range(1, math.ceil(1/pp3) + 1):
                if iii*pp3 < 1:
                    cut3 = np.quantile(X3, [(iii-1)*pp3, iii*pp3])
                else:
                    cut3 = np.quantile(X3, [(iii-1)*pp3, 1])

                addon = addont
                
                IND = np.array([], dtype=int)
                if len(X1) > min_loc:
                    # Increase addon until we have enough indices
                    while len(IND) < min_loc:
                        addon = addon + 10
                        IND = np.where((X1 > (cut1[0] - addon)) & (X1 < (cut1[1] + addon)) & \
                                        (X2 > (cut2[0] - addon)) & (X2 < (cut2[1] + addon)) & \
                                        (X3 >= (cut3[0] - addon)) & (X3 <= (cut3[1] + addon)))[0]
                else:
                    IND = np.arange(len(X1))
                print(IND[0:10])
                while len(IND) > min_loc:
                    addon = addon - 10
                    IND = np.where((X1 > (cut1[0] - addon)) & (X1 < (cut1[1] + addon)) & \
                                    (X2 > (cut2[0] - addon)) & (X2 < (cut2[1] + addon)) & \
                                    (X3 >= (cut3[0] - addon)) & (X3 <= (cut3[1] + addon)))[0]
                    if addon < 150:
                        break
                    
                if (len(TrueLocalizations) != 0) and (isinstance(TrueLocalizations[ksu], np.ndarray)) and (TrueLocalizations[ksu].size != 0):
                    INDt = np.where((X1t > (cut1[0] - addon)) & (X1t < (cut1[1] + addon)) & \
                                    (X2t > (cut2[0] - addon)) & (X2t < (cut2[1] + addon)) & \
                                    (X3t >= (cut3[0] - addon)) & (X3t <= (cut3[1] + addon)))[0]

                # Visualize
                # try:
                #     plt.figure(1, figsize=(8, 6))
                #     plt.clf()  # Clear figure first

                #     if len(IND) > 0 and len(X4) >= max(IND) + 1:
                #         c_values = X4[IND]
                #     else:
                #         c_values = np.ones(len(IND))
                #         print("Using default colors for visualization")

                #     if np.all(X3[IND] == 0):  # 2D plot
                #         sc = plt.scatter(X1[IND], X2[IND], s=10, c=c_values, cmap='jet')
                #         plt.axis('equal')
                #         plt.colorbar(sc)
                #     else:  # 3D plot
                #         ax = plt.axes(projection='3d')
                #         sc = ax.scatter(X1[IND], X2[IND], X3[IND], s=10, c=c_values, cmap='jet')
                #         plt.colorbar(sc)
                #         plt.tight_layout()

                #     plt.title(f"Split {counter+1}: {len(IND)} points")
                #     plt.draw()
                #     plt.pause(0.5)
                # except Exception as e:
                #     print(f"Warning: Failed to create plot: {e}")
                print(IND)

                # Append split data
                LF_split = np.column_stack((X1[IND], X2[IND], X3[IND]))
                LocalizationsFinal_Split.append(LF_split)
                Photons_Split.append(Photons[ksu][IND])
                
                if (len(TrueLocalizations) != 0) and (isinstance(TrueLocalizations[ksu], np.ndarray)) and (TrueLocalizations[ksu].size != 0):
                    TL_split = np.column_stack((X1t[INDt], X2t[INDt], X3t[INDt]))
                    TrueLocalizations_Split.append(TL_split)
                else:
                    TrueLocalizations_Split.append(np.array([]))
                
                cut1array.append(cut1)
                cut2array.append(cut2)
                cut3array.append(cut3)
                # For Frame_Information, select indices
                X4_ind = X4[IND]
                Frame_Information_Split.append(X4_ind)
                Came_from_image.append(ksu)
                Parameters_to_split.append([ppf, ppf2, ppf3])
                addonarray.append(addon)
                temp_numb_of_loc.append(len(IND))
                
                #This will account for images that allready have less than
                #the need number of localizations.
                if len(X1) < min_loc:
                    flag = 1
                    break


Processing frame 1/6
[   0   25   26 ... 3975 3976 3987]
[   0    1    2 ... 3984 3986 3987]
[   1    2    3 ... 3983 3984 3985]
[   0    1    2 ... 3985 3986 3987]
[   1    2    3 ... 3997 3998 3999]
[   1    2    3 ... 3987 3994 3997]
[   1    2    3 ... 3997 3998 3999]
[   1    2    3 ... 3997 3998 3999]
[   1    2    3 ... 3997 3998 3999]
[   1    2    3 ... 3997 3998 3999]
Processing frame 2/6
[   0    1    2 ... 3996 3997 3998]
[   0    1    2 ... 3996 3997 3998]
[   0    1    2 ... 3996 3997 3998]
[   0    1    2 ... 3996 3997 3998]
[   5    6    7 ... 3996 3997 3999]
[  14   15   16 ... 3996 3997 3998]
[   5    6   31 ... 3996 3998 3999]
[  18   20   33 ... 3996 3998 3999]
[  18   46   47 ... 3996 3998 3999]
[  18   20   33 ... 3996 3998 3999]


In [16]:
matlab_result = sio.loadmat('Split_simple2.mat', squeeze_me=True, struct_as_record=False)

In [29]:
print(matlab_result['Frame_Information'][0])
print(Frame_Information_Split[0])

[ 461 9457 9458 ... 1729 1730 3177]
[5429 5438 5456 ... 4229 6556 6558]



Choose one mainloop:
1. Adding data through append
2. Adding data in Matlab style


First case : Adding data through append

In [ ]:
# Main processing loop
counter = 0
for ksu in range(len(Frame_Information)):
    print(f"Processing frame {ksu+1}/{len(Frame_Information)}")
    
    # Get the localization data
    LF = LocalizationsFinal[ksu]
    
    # Extract columns - now we know LF has 3 columns
    X1, X2, X3 = LF[:,0], LF[:,1], LF[:,2]
    
    # Handle TrueLocalizations for this frame
    has_true_loc = (ksu < len(TrueLocalizations) and 
                    TrueLocalizations[ksu].size > 0)
    
    if has_true_loc:
        TL = TrueLocalizations[ksu]
        X1t, X2t, X3t = TL[:,0], TL[:,1], TL[:,2]
    
    X4 = Frame_Information[ksu].copy()
    addon = 250  # Buffer region
    
    # Phase space search
    scorefinal = np.inf
    onwers = 1

    while onwers < 300:
        print(f'Phase space search progress: {onwers/300:.2f}')
        temp_numb_of_loc = []
        onwers += 1
        random.seed(10)
        pp = random.random()*0.95 + 0.05
        pp2 = random.random()*0.95 + 0.05
        pp3 = random.random()*0.95 + 0.05 if np.max(X3) != 0 else 1

        # Loop over quantile partitions
        for i in range(1, math.ceil(1/pp)+1):
            if i*pp < 1:
                cut1 = np.quantile(X1, [(i-1)*pp, i*pp])
            else:
                cut1 = np.quantile(X1, [(i-1)*pp, 1])
            
            for ii in range(1, math.ceil(1/pp2)+1):      
                if ii*pp2 < 1:
                    cut2 = np.quantile(X2, [(ii-1)*pp2, ii*pp2])
                else:
                    cut2 = np.quantile(X2, [(ii-1)*pp2, 1])
                
                for iii in range(1, math.ceil(1/pp3)+1):
                    if iii*pp3 < 1:
                        cut3 = np.quantile(X3, [(iii-1)*pp3, iii*pp3])
                    else:
                        cut3 = np.quantile(X3, [(iii-1)*pp3, 1])
                    
                    # Find indices with conditions and buffer addon
                    IND = np.where((X1 >= (cut1[0] - addon)) & (X1 <= (cut1[1] + addon)) & 
                                   (X2 >= (cut2[0] - addon)) & (X2 <= (cut2[1] + addon)) & 
                                   (X3 >= (cut3[0] - addon)) & (X3 <= (cut3[1] + addon)))[0]
                    temp_numb_of_loc.append(len(IND))
        
        # Scoring function
        if not temp_numb_of_loc:  # Handle empty list
            continue
            
        # Calculate score
        Resid = np.array(temp_numb_of_loc) - min_loc
        score = np.sum(Resid**2) / len(temp_numb_of_loc)
        
        if score < scorefinal:
            scorefinal = score
            ppf, ppf2, ppf3 = pp, pp2, pp3
            onwers = -onwers  # Reset counter
            temp_numb_of_locf = temp_numb_of_loc.copy()
    
    addont = addon
    
    # Set parameters based on results
    pp, pp2, pp3 = ppf, ppf2, ppf3
    
    if len(X1) < min_loc:
        ppf = pp = ppf2 = pp2 = ppf3 = pp3 = 1
    
    flag = 0


    #Now that we know the optimum way to split up our specific image, we
    #now split it up. 
    pp = ppf
    pp2 = ppf2
    pp3 = ppf3

    if len(X1) < min_loc:
        ppf = 1
        ppf2 = 1
        ppf3 = 1
        pp = ppf
        pp2 = ppf2
        pp3 = ppf3

    flag = 0

    for i in range(1, math.ceil(1/pp) + 1):
        if flag == 1:
            break
        
        if i*pp < 1:
            cut1 = np.quantile(X1, [(i-1)*pp, i*pp])
        else:
            cut1 = np.quantile(X1, [(i-1)*pp, 1])
        
        for ii in range(1, math.ceil(1/pp2) + 1):
            if ii*pp2 < 1:
                cut2 = np.quantile(X2, [(ii-1)*pp2, ii*pp2])
            else:
                cut2 = np.quantile(X2, [(ii-1)*pp2, 1])
            
            for iii in range(1, math.ceil(1/pp3) + 1):
                if iii*pp3 < 1:
                    cut3 = np.quantile(X3, [(iii-1)*pp3, iii*pp3])
                else:
                    cut3 = np.quantile(X3, [(iii-1)*pp3, 1])

                addon = addont
                
                IND = np.array([], dtype=int)
                if len(X1) > min_loc:
                    # Increase addon until we have enough indices
                    while len(IND) < min_loc:
                        addon = addon + 10
                        IND = np.where((X1 > (cut1[0] - addon)) & (X1 < (cut1[1] + addon)) & \
                                        (X2 > (cut2[0] - addon)) & (X2 < (cut2[1] + addon)) & \
                                        (X3 >= (cut3[0] - addon)) & (X3 <= (cut3[1] + addon)))[0]
                else:
                    IND = np.arange(len(X1))
                
                while len(IND) > min_loc:
                    addon = addon - 10
                    IND = np.where((X1 > (cut1[0] - addon)) & (X1 < (cut1[1] + addon)) & \
                                    (X2 > (cut2[0] - addon)) & (X2 < (cut2[1] + addon)) & \
                                    (X3 >= (cut3[0] - addon)) & (X3 <= (cut3[1] + addon)))[0]
                    if addon < 150:
                        break
                    
                if (len(TrueLocalizations) != 0) and (isinstance(TrueLocalizations[ksu], np.ndarray)) and (TrueLocalizations[ksu].size != 0):
                    INDt = np.where((X1t > (cut1[0] - addon)) & (X1t < (cut1[1] + addon)) & \
                                    (X2t > (cut2[0] - addon)) & (X2t < (cut2[1] + addon)) & \
                                    (X3t >= (cut3[0] - addon)) & (X3t <= (cut3[1] + addon)))[0]

                # Visualize
                try:
                    plt.figure(1, figsize=(8, 6))
                    plt.clf()  # Clear figure first

                    if len(IND) > 0 and len(X4) >= max(IND) + 1:
                        c_values = X4[IND]
                    else:
                        c_values = np.ones(len(IND))
                        print("Using default colors for visualization")

                    if np.all(X3[IND] == 0):  # 2D plot
                        sc = plt.scatter(X1[IND], X2[IND], s=10, c=c_values, cmap='jet')
                        plt.axis('equal')
                        plt.colorbar(sc)
                    else:  # 3D plot
                        ax = plt.axes(projection='3d')
                        sc = ax.scatter(X1[IND], X2[IND], X3[IND], s=10, c=c_values, cmap='jet')
                        plt.colorbar(sc)
                        plt.tight_layout()

                    plt.title(f"Split {counter+1}: {len(IND)} points")
                    plt.draw()
                    plt.pause(0.5)
                except Exception as e:
                    print(f"Warning: Failed to create plot: {e}")
                
                # Append split data
                LF_split = np.column_stack((X1[IND], X2[IND], X3[IND]))
                LocalizationsFinal_Split.append(LF_split)
                Photons_Split.append(Photons[ksu][IND])
                
                if (len(TrueLocalizations) != 0) and (isinstance(TrueLocalizations[ksu], np.ndarray)) and (TrueLocalizations[ksu].size != 0):
                    TL_split = np.column_stack((X1t[INDt], X2t[INDt], X3t[INDt]))
                    TrueLocalizations_Split.append(TL_split)
                else:
                    TrueLocalizations_Split.append(np.array([]))
                
                cut1array.append(cut1)
                cut2array.append(cut2)
                cut3array.append(cut3)
                # For Frame_Information, select indices
                X4_ind = X4[IND]
                Frame_Information_Split.append(X4_ind)
                Came_from_image.append(ksu)
                Parameters_to_split.append([ppf, ppf2, ppf3])
                addonarray.append(addon)
                temp_numb_of_loc.append(len(IND))
                
                #This will account for images that allready have less than
                #the need number of localizations.
                if len(X1) < min_loc:
                    flag = 1
                    break


Second case : Another approach Matlab like by assigning counter for each split.

In [ ]:
# Before main loop, initialize lists with sufficient length
max_possible_splits = 10000000  # Or some conservative upper bound
LocalizationsFinal_Split = [None] * max_possible_splits
Photons_Split = [None] * max_possible_splits
TrueLocalizations_Split = [None] * max_possible_splits
cut1array = [None] * max_possible_splits
cut2array = [None] * max_possible_splits
cut3array = [None] * max_possible_splits
Frame_Information_Split = [None] * max_possible_splits
Came_from_image = [None] * max_possible_splits
Parameters_to_split = [None] * max_possible_splits
addonarray = [None] * max_possible_splits

# Main processing loop
counter = 0
for ksu in range(len(Frame_Information)):
    print(f"Processing frame {ksu+1}/{len(Frame_Information)}")
    
    # Get the localization data
    LF = LocalizationsFinal[ksu]
    
    # Extract columns - now we know LF has 3 columns
    X1, X2, X3 = LF[:,0], LF[:,1], LF[:,2]
    
    # Handle TrueLocalizations for this frame
    has_true_loc = (ksu < len(TrueLocalizations) and 
                    TrueLocalizations[ksu].size > 0)
    
    if has_true_loc:
        TL = TrueLocalizations[ksu]
        X1t, X2t, X3t = TL[:,0], TL[:,1], TL[:,2]
    
    X4 = Frame_Information[ksu].copy()
    addon = 250  # Buffer region
    
    # Phase space search
    scorefinal = np.inf
    onwers = 1
    while onwers < 300:
        print(f'Phase space search progress: {onwers/300:.2f}')
        temp_numb_of_loc = []
        onwers += 1
        pp = random.random()*0.95 + 0.05
        pp2 = random.random()*0.95 + 0.05
        pp3 = random.random()*0.95 + 0.05 if np.max(X3) != 0 else 1

        # Loop over quantile partitions
        for i in range(1, math.ceil(1/pp)+1):
            if i*pp < 1:
                cut1 = np.quantile(X1, [(i-1)*pp, i*pp])
            else:
                cut1 = np.quantile(X1, [(i-1)*pp, 1])
            
            for ii in range(1, math.ceil(1/pp2)+1):      
                if ii*pp2 < 1:
                    cut2 = np.quantile(X2, [(ii-1)*pp2, ii*pp2])
                else:
                    cut2 = np.quantile(X2, [(ii-1)*pp2, 1])
                
                for iii in range(1, math.ceil(1/pp3)+1):
                    if iii*pp3 < 1:
                        cut3 = np.quantile(X3, [(iii-1)*pp3, iii*pp3])
                    else:
                        cut3 = np.quantile(X3, [(iii-1)*pp3, 1])
                    
                    # Find indices with conditions and buffer addon
                    IND = np.where((X1 >= (cut1[0] - addon)) & (X1 <= (cut1[1] + addon)) & 
                                   (X2 >= (cut2[0] - addon)) & (X2 <= (cut2[1] + addon)) & 
                                   (X3 >= (cut3[0] - addon)) & (X3 <= (cut3[1] + addon)))[0]
                    temp_numb_of_loc.append(len(IND))
        
        # Scoring function
        if not temp_numb_of_loc:  # Handle empty list
            continue
            
        # Calculate score
        Resid = np.array(temp_numb_of_loc) - min_loc
        score = np.sum(Resid**2) / len(temp_numb_of_loc)
        
        if score < scorefinal:
            scorefinal = score
            ppf, ppf2, ppf3 = pp, pp2, pp3
            onwers = -onwers  # Reset counter
            temp_numb_of_locf = temp_numb_of_loc.copy()
    
    addont = addon
    
    # Set parameters based on results
    pp, pp2, pp3 = ppf, ppf2, ppf3
    
    if len(X1) < min_loc:
        ppf = pp = ppf2 = pp2 = ppf3 = pp3 = 1
    
    flag = 0


    #Now that we know the optimum way to split up our specific image, we
    #now split it up. 
    ##

    pp = ppf
    pp2 = ppf2
    pp3 = ppf3

    if len(X1) < min_loc:
        ppf = 1
        ppf2 = 1
        ppf3 = 1
        pp = ppf
        pp2 = ppf2
        pp3 = ppf3

    flag = 0

    for i in range(1, math.ceil(1/pp) + 1):
        if flag == 1:
            break
        
        if i*pp < 1:
            cut1 = np.quantile(X1, [(i-1)*pp, i*pp])
        else:
            cut1 = np.quantile(X1, [(i-1)*pp, 1])
        
        for ii in range(1, math.ceil(1/pp2) + 1):
            if ii*pp2 < 1:
                cut2 = np.quantile(X2, [(ii-1)*pp2, ii*pp2])
            else:
                cut2 = np.quantile(X2, [(ii-1)*pp2, 1])
            
            for iii in range(1, math.ceil(1/pp3) + 1):
                if iii*pp3 < 1:
                    cut3 = np.quantile(X3, [(iii-1)*pp3, iii*pp3])
                else:
                    cut3 = np.quantile(X3, [(iii-1)*pp3, 1])

                addon = addont
                
                IND = np.array([], dtype=int)
                if len(X1) > min_loc:
                    # Increase addon until we have enough indices
                    while len(IND) < min_loc:
                        addon = addon + 10
                        IND = np.where((X1 > (cut1[0] - addon)) & (X1 < (cut1[1] + addon)) & \
                                        (X2 > (cut2[0] - addon)) & (X2 < (cut2[1] + addon)) & \
                                        (X3 >= (cut3[0] - addon)) & (X3 <= (cut3[1] + addon)))[0]
                else:
                    IND = np.arange(len(X1))
                
                while len(IND) > min_loc:
                    addon = addon - 10
                    IND = np.where((X1 > (cut1[0] - addon)) & (X1 < (cut1[1] + addon)) & \
                                    (X2 > (cut2[0] - addon)) & (X2 < (cut2[1] + addon)) & \
                                    (X3 >= (cut3[0] - addon)) & (X3 <= (cut3[1] + addon)))[0]
                    if addon < 150:
                        break
                    
                if (len(TrueLocalizations) != 0) and (isinstance(TrueLocalizations[ksu], np.ndarray)) and (TrueLocalizations[ksu].size != 0):
                    INDt = np.where((X1t > (cut1[0] - addon)) & (X1t < (cut1[1] + addon)) & \
                                    (X2t > (cut2[0] - addon)) & (X2t < (cut2[1] + addon)) & \
                                    (X3t >= (cut3[0] - addon)) & (X3t <= (cut3[1] + addon)))[0]
                    
                # Visualize
                try:
                    plt.figure(1, figsize=(8, 6))
                    plt.clf()  # Clear figure first

                    if len(IND) > 0 and len(X4) >= max(IND) + 1:
                        c_values = X4[IND]
                    else:
                        c_values = np.ones(len(IND))
                        print("Using default colors for visualization")

                    if np.all(X3[IND] == 0):  # 2D plot
                        sc = plt.scatter(X1[IND], X2[IND], s=10, c=c_values, cmap='jet')
                        plt.axis('equal')
                        plt.colorbar(sc)
                    else:  # 3D plot
                        ax = plt.axes(projection='3d')
                        sc = ax.scatter(X1[IND], X2[IND], X3[IND], s=10, c=c_values, cmap='jet')
                        plt.colorbar(sc)
                        plt.tight_layout()

                    plt.title(f"Split {counter+1}: {len(IND)} points")
                    plt.draw()
                    plt.pause(0.5)
                except Exception as e:
                    print(f"Warning: Failed to create plot: {e}")
                
                # Append split data
                LF_split = np.column_stack((X1[IND], X2[IND], X3[IND]))
                LocalizationsFinal_Split[counter] = LF_split
                Photons_Split[counter] = Photons[ksu][IND]

                if has_true_loc:
                    TL_split = np.column_stack((X1t[INDt], X2t[INDt], X3t[INDt]))
                    TrueLocalizations_Split[counter] = TL_split
                else:
                    TrueLocalizations_Split[counter] = np.array([])

                cut1array[counter] = cut1
                cut2array[counter] = cut2
                cut3array[counter] = cut3
                Frame_Information_Split[counter] = X4[IND]
                Came_from_image[counter] = ksu
                Parameters_to_split[counter] = [ppf, ppf2, ppf3]
                addonarray[counter] = addon

                counter += 1 
                
                #This will account for images that allready have less than
                #the need number of localizations.
                if len(X1) < min_loc:
                    flag = 1
                    break

    LocalizationsFinal_Split = LocalizationsFinal_Split[:counter]
    Photons_Split = Photons_Split[:counter]
    TrueLocalizations_Split = TrueLocalizations_Split[:counter]
    cut1array = cut1array[:counter]
    cut2array = cut2array[:counter]
    cut3array = cut3array[:counter]
    Frame_Information_Split = Frame_Information_Split[:counter]
    Came_from_image = Came_from_image[:counter]
    Parameters_to_split = Parameters_to_split[:counter]
    addonarray = addonarray[:counter]

Finally we update the splitted data and save it

In [ ]:
# Update with split data
LocalizationsFinal = LocalizationsFinal_Split
Frame_Information = Frame_Information_Split
TrueLocalizations = TrueLocalizations_Split
Photons = Photons_Split

print(f"Created {len(LocalizationsFinal)} split images")

# Prepare data for saving
# Convert lists to numpy arrays for proper saving
addonarray_np = np.array(addonarray)
Came_from_image_np = np.array(Came_from_image)

# Prepare dictionary to save
save_dict = {
    'LocalizationsFinal': np.array(LocalizationsFinal, dtype=object),
    'Final_Loc_Blinking_Corrected': np.array(LocalizationsFinal, dtype=object),
    'Final_Frame_Blinking_Corrected': np.array(Frame_Information, dtype=object),
    'Frame_Information': np.array(Frame_Information, dtype=object),
    'addonarray': addonarray_np,
    'Parameters_to_split': np.array(Parameters_to_split, dtype=object),
    'Came_from_image': Came_from_image_np,
    'TrueLocalizations': np.array(TrueLocalizations, dtype=object),
    'cut1array': np.array(cut1array, dtype=object),
    'cut2array': np.array(cut2array, dtype=object),
    'cut3array': np.array(cut3array, dtype=object),
    'Resolution': Resolution,
    'Photons': np.array(Photons, dtype=object)
}

# Save the results
output_filename = 'Split3_' + Condition
try:
    sio.savemat(output_filename, save_dict)
    print(f'Processing complete. File saved as: {output_filename}')
except Exception as e:
    print(f"Error saving output file: {e}")
    # Backup save attempt with simpler structure
    try:
        simple_dict = {k: v for k, v in save_dict.items() if isinstance(v, (np.ndarray, list))}
        sio.savemat(output_filename + "_backup", simple_dict)
        print(f"Saved backup file with simpler structure: {output_filename}_backup")
    except:
        print("Failed to save backup file")

# Clean up
plt.close('all')
print("Script completed successfully")